In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

# CSV 파일 읽어오기
circulatory_df = pd.read_csv('content/circulatory_diseases_combined.csv')
patients_df = pd.read_csv('content/patients.csv')

# subject_id를 기준으로 두 데이터프레임을 병합
merged_df = pd.merge(circulatory_df, patients_df, on='subject_id', how='inner')

# 병합된 데이터를 patients_icu.csv 파일로 저장
merged_df.to_csv('content/patients_icu.csv', index=False)

In [4]:
# 데이터 삽입
chartevents = pd.read_csv("content/12hour_chartevents.csv")
#inputevents = pd.read_csv("content/6hour_inputevents.csv")
#outputevents = pd.read_csv("content/6hour_outputevents.csv")
#procedureevents = pd.read_csv("content/6hour_procedureevents.csv")
patients = pd.read_csv("content/patients_icu.csv") #순환기 환자 기준
d_items = pd.read_csv("content/d_items.csv")
icustays = pd.read_csv("content/icustays.csv")
diagnoses = pd.read_csv("content/diagnoses_icd.csv")
d_icd = pd.read_csv("content/d_icd_diagnoses.csv")

print(chartevents.shape)
#print(inputevents.shape)
#print(outputevents.shape)
#print(patients.shape)
print(d_items.shape)
print(icustays.shape)
print(diagnoses.shape)
print(d_icd.shape)
print(patients)

C:\Users\Eugene Clark\AppData\Local\Temp\ipykernel_17396\3020803163.py:2: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.read_csv("content/12hour_chartevents.csv")


(52835723, 12)
(3861, 9)
(76540, 8)
(5280351, 5)
(109775, 3)
       subject_id   hadm_id   stay_id  \
0        16256226  20013290  39289362   
1        19194449  21641999  39387567   
2        12974563  29618057  32563675   
3        14311522  24622512  30002548   
4        12509799  25897223  30004530   
...           ...       ...       ...   
26124    11414090  25441049  39987151   
26125    19127408  24375536  39988534   
26126    16196296  29925962  39988873   
26127    12275003  22562812  39992247   
26128    17840864  22695803  39999810   

                                     first_careunit  \
0                                    Neuro Stepdown   
1                          Coronary Care Unit (CCU)   
2                                    Neuro Stepdown   
3      Cardiac Vascular Intensive Care Unit (CVICU)   
4      Cardiac Vascular Intensive Care Unit (CVICU)   
...                                             ...   
26124                      Coronary Care Unit (CCU)   
26125 

In [5]:
chartevents_m = pd.merge(chartevents,d_items, on = 'itemid')
#inputevents_m = pd.merge(inputevents,d_items, on = 'itemid')
#outputevents_m = pd.merge(outputevents,d_items, on = 'itemid')
#이벤트의 이름 확인을 위해 d_items와 병합

In [6]:
# 화씨를 측정한 이벤트는 섭씨로 변환함 -> 섭씨를 측정한 이벤트 사용
chartevents_m.loc[chartevents_m['label'] == 'Temperature Fahrenheit','valuenum'] = chartevents_m.loc[chartevents_m['label'] == 'Temperature Fahrenheit','valuenum'].map(lambda x : (x - 32) * 5.0 / 9.0)
chartevents_m.loc[chartevents_m['label'] == 'Temperature Fahrenheit','valuenum']
# 아래는 변환 결과

44          36.555556
1475        36.611111
1556        36.611111
1615        36.666667
1647        36.888889
              ...    
48421094    37.055556
48421129    36.055556
48421182    35.555556
48421242    35.722222
48421290    35.777778
Name: valuenum, Length: 254975, dtype: float64

In [7]:
chart = chartevents_m[['subject_id','stay_id','charttime','itemid','value','valuenum',"valueuom",'intime','label']]
chart.to_csv('content/circulator_chart_12.csv', index=False)
print(chart)

          subject_id   stay_id            charttime  itemid  \
0           10003700  30600691  2165-04-24 06:00:00  220045   
1           10003700  30600691  2165-04-24 06:00:00  220179   
2           10003700  30600691  2165-04-24 06:00:00  220180   
3           10003700  30600691  2165-04-24 06:00:00  220181   
4           10003700  30600691  2165-04-24 06:00:00  220210   
...              ...       ...                  ...     ...   
52835718    19999068  30143796  2161-08-24 17:00:00  223820   
52835719    19999068  30143796  2161-08-24 17:00:00  223821   
52835720    19999068  30143796  2161-08-24 17:00:00  223823   
52835721    19999068  30143796  2161-08-24 17:00:00  223824   
52835722    19999068  30143796  2161-08-24 17:00:00  224650   

                                     value  valuenum  valueuom  \
0                                     56.0      56.0       bpm   
1                                    126.0     126.0      mmHg   
2                                     78.0   

In [8]:
import pandas as pd

# 1. patients_icu에서 hadm_id를 추출
icu_hadm_ids = patients['hadm_id']

# 2. icustays 데이터에서 hadm_id가 patients_icu에 있는 hadm_id만 필터링
filtered_icustays = icustays[icustays['hadm_id'].isin(icu_hadm_ids)]

# 3. los 컬럼에 대한 요약 통계 출력
filtered_icustays['los'].describe()

count    26129.000000
mean         3.392605
std          4.683605
min          0.001250
25%          1.155903
50%          1.997662
75%          3.684537
max        103.499005
Name: los, dtype: float64

In [10]:
icustays = icustays[~((icustays['los'] >  21.07) | (icustays['los'] < 0.5))]
stay_id = icustays['stay_id']
#6시간보다 빨리 퇴원하거나, 입실기간이 2표준편차 이상인 이상치 제거
chart = chart[chart['stay_id'].isin(stay_id)]
chart
chart.to_csv('content/circulator_chart_12.csv', index=False)

In [11]:
import pandas as pd
chart = pd.read_csv('content/circulator_chart_12.csv')

C:\Users\Eugene Clark\AppData\Local\Temp\ipykernel_17396\1473327434.py:2: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  chart = pd.read_csv('content/circulator_chart_12.csv')


In [12]:
# 'label' 컬럼에서 결측치를 제외하고 유니크 값들의 빈도수를 계산하고 상위 20개를 나열
top_20_labels = chart['label'].value_counts(dropna=True).head(50)

# 결과 출력
print(top_20_labels)

label
Safety Measures                                  1393964
Heart Rate                                       1086915
Respiratory Rate                                 1075466
O2 saturation pulseoxymetry                      1060703
Heart Rhythm                                      897828
Ectopy Type 1                                     767144
Non Invasive Blood Pressure systolic              718869
Non Invasive Blood Pressure diastolic             718694
Non Invasive Blood Pressure mean                  718339
Orientation                                       424523
Less Restrictive Measures                         377905
Turn                                              350562
Head of Bed                                       347141
Arterial Blood Pressure mean                      337085
Arterial Blood Pressure systolic                  335124
Arterial Blood Pressure diastolic                 335063
Position                                          320883
Activity Tolerance       

In [14]:
# 해당 라벨값의 행 조회
chart[(chart['label']=='Heart Rhythm')].head

<bound method NDFrame.head of           subject_id   stay_id            charttime  itemid  \
1380278     10004235  34100191  2196-02-24 17:16:00  220048   
1380342     10004235  34100191  2196-02-24 18:00:00  220048   
1380426     10004235  34100191  2196-02-24 19:00:00  220048   
1380442     10004235  34100191  2196-02-24 20:00:00  220048   
1380585     10004235  34100191  2196-02-24 21:00:00  220048   
...              ...       ...                  ...     ...   
29979648    19999068  30143796  2161-08-24 07:00:00  220048   
29979666    19999068  30143796  2161-08-24 08:00:00  220048   
29979800    19999068  30143796  2161-08-24 09:00:00  220048   
29979817    19999068  30143796  2161-08-24 10:00:00  220048   
29979851    19999068  30143796  2161-08-24 11:00:00  220048   

                             value  valuenum valueuom               intime  \
1380278   AF (Atrial Fibrillation)       NaN      NaN  2196-02-24 17:07:00   
1380342   AF (Atrial Fibrillation)       NaN      NaN  21

In [13]:
import pandas as pd

# 이벤트 그룹 정의 -> 각 측정할 값에 해당하는 이벤트들
hr_events = ["Heart Rate", "Heart rate Alarm - High", "Heart Rate Alarm - Low"]
systolic_bp_events = ["Arterial Blood Pressure systolic", "ART BP Systolic", "Non Invasive Blood Pressure systolic"]
diastolic_bp_events = ['Arterial Blood Pressure diastolic', 'ART BP Diastolic', 'Non Invasive Blood Pressure diastolic']
Blood_Glucose = ["Glucose (serum)", "Glucose (whole blood)", 'Glucose finger stick (range 70-100)']
O2_Saturation_Pulseoxymetry = ["O2 saturation pulseoxymetry"]
temperature = ["Temperature Fahrenheit", "Temperature Celsius"]
Respiratory_Rate = ['Respiratory Rate']
PH = ['PH (Arterial)', 'PH (Venous)']
GCS_Eye = ['GCS - Eye Opening']
GCS_Verbal = ['GCS - Verbal Response']
GCS_Motor = ['GCS - Motor Response']
Arterial_Blood_Pressure_mean = ['Arterial Blood Pressure mean']
#i1 = ['Arterial Blood Pressure mean']
#i2 = 'Arterial Blood Pressure mean'

# 범주형 변수 이벤트 그룹
categorical_events = ['Pain Assessment Method', 'Activity Tolerance', 'Head of Bed']

# 필터링할 이벤트만 있는 행 선택
filtered_df = chart[chart['label'].isin(hr_events + systolic_bp_events + diastolic_bp_events +
                                        Blood_Glucose + O2_Saturation_Pulseoxymetry + temperature + Respiratory_Rate + 
                                        PH + GCS_Eye + GCS_Verbal + GCS_Motor + categorical_events)].copy()

# intime 컬럼을 datetime 형식으로 변환 -> 시간 계산을 위해 변경
filtered_df['intime'] = pd.to_datetime(filtered_df['intime'])
filtered_df['charttime'] = pd.to_datetime(filtered_df['charttime'])

# 입실 후 시간 단위로 구분 -> 각 이벤트들을 1시간동안 이벤트인지 2시간 동안 이벤트인지 설정하여 새로운 컬럼으로 만듬
filtered_df['time_since_admission'] = (filtered_df['charttime'] - filtered_df['intime']).dt.total_seconds() / 3600
filtered_df['time_bin'] = filtered_df['time_since_admission'].apply(lambda x: int(x) if x < 12 else 11)  # 0~5시간까지만 포함

# 이벤트 그룹별 레이블 지정 함수
def get_event_group(label):
    if label in hr_events:
        return 'Heart Rate'
    elif label in systolic_bp_events:
        return 'Systolic BP'
    elif label in diastolic_bp_events:
        return 'Diastolic BP'
    elif label in Blood_Glucose:
        return 'Blood Glucose'
    elif label in O2_Saturation_Pulseoxymetry:
        return 'O2 Saturation Pulseoxymetry'
    elif label in temperature:
        return 'Temperature'
    elif label in Respiratory_Rate:
        return 'Respiratory Rate'
    elif label in PH:
        return 'Arterial PH'
    elif label in GCS_Eye:
        return 'GCS_Eye'
    elif label in GCS_Verbal:
        return 'GCS_Verbal'
    elif label in GCS_Motor:
        return 'GCS_Motor'
    elif label in categorical_events:
        return label
    else:
        return 'Other'

# 그룹으로 분류하여 새로운 컬럼으로 만듦
filtered_df['event_group'] = filtered_df['label'].apply(get_event_group)

# 수치형 변수 처리
numerical_events = hr_events + systolic_bp_events + diastolic_bp_events + Blood_Glucose + O2_Saturation_Pulseoxymetry + temperature + Respiratory_Rate + PH + GCS_Eye + GCS_Verbal + GCS_Motor

numerical_df = filtered_df[filtered_df['label'].isin(numerical_events)].copy()

# 범주형 변수 처리
categorical_df = filtered_df[filtered_df['label'].isin(categorical_events)].copy()

# 수치형 변수에 대해 평균 계산
numerical_pivot = (numerical_df.groupby(['stay_id', 'time_bin', 'event_group'])
                   .agg({'valuenum': 'mean'})
                   .reset_index()
                   .pivot(index=['stay_id', 'time_bin'], columns='event_group', values='valuenum')
                   .reset_index())

# 범주형 변수에서 최빈값 계산 (각 레이블에 맞게 컬럼명 지정)
def get_mode(df, col, value_col):
    """범주형 변수에서 최빈값을 계산"""
    mode_df = df[df['label'] == col].groupby(['stay_id', 'time_bin'])[value_col] \
        .agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()

    # 최빈값이 계산된 컬럼을 레이블로 변경
    mode_df = mode_df.rename(columns={value_col: col})
    
    return mode_df

# 각 범주형 변수에 대해 최빈값을 계산하고 병합
mode_dfs = []
for categorical in categorical_events:
    mode_df = get_mode(filtered_df, categorical, 'value')
    mode_dfs.append(mode_df)

# 모든 범주형 데이터 최빈값을 하나의 데이터프레임으로 결합
mode_combined_df = mode_dfs[0]
for df in mode_dfs[1:]:
    mode_combined_df = pd.merge(mode_combined_df, df, on=['stay_id', 'time_bin'], how='left')

# 수치형 데이터와 범주형 데이터 병합
final_df = pd.merge(numerical_pivot, mode_combined_df, on=['stay_id', 'time_bin'], how='left')

# 결과 확인
final_df


,stay_id,time_bin,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed
0,30000153,0,NaN,NaN,77.0,3.0,5.0,1.0,104.000000,100.0,17.0,113.0,NaN,Change in Vitals,Tolerated Well,Reverse T-berg
1,30000153,1,7.30,167.0,66.5,NaN,NaN,NaN,NaN,NaN,16.0,141.0,37.277778,NaN,NaN,NaN
2,30000153,2,NaN,NaN,NaN,NaN,NaN,NaN,83.000000,100.0,NaN,NaN,NaN,NaN,NaN,NaN
3,30000153,3,7.31,183.5,60.0,NaN,NaN,NaN,87.500000,100.0,15.0,116.0,37.500000,NaN,NaN,NaN
4,30000153,4,NaN,NaN,56.0,4.0,6.0,1.0,103.000000,100.0,20.0,111.0,NaN,Non-verbal Cues,Tolerated Well,Reverse T-berg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833000,39999810,7,NaN,120.0,84.0,NaN,NaN,NaN,78.666667,98.0,16.0,129.0,36.777778,NaN,NaN,NaN
833001,39999810,8,NaN,NaN,NaN,3.0,6.0,5.0,NaN,NaN,NaN,NaN,NaN,Patient Verbalized,Tolerated Well,30 Degrees
833002,39999810,9,NaN,NaN,101.0,4.0,6.0,5.0,72.000000,98.0,17.0,170.0,NaN,NaN,NaN,NaN
833003,39999810,10,NaN,NaN,97.0,NaN,NaN,NaN,NaN,NaN,NaN,165.0,NaN,NaN,NaN,NaN


In [14]:
final_df.to_csv('final_df_v1_dataset.csv_12', index=False, encoding='utf-8-sig')

In [15]:
#결측치 비율 출력
final_df.isnull().sum() / len(final_df)

stay_id                        0.000000
time_bin                       0.000000
Arterial PH                    0.876431
Blood Glucose                  0.725758
Diastolic BP                   0.065587
GCS_Eye                        0.661288
GCS_Motor                      0.663032
GCS_Verbal                     0.661942
Heart Rate                     0.037038
O2 Saturation Pulseoxymetry    0.053734
Respiratory Rate               0.047009
Systolic BP                    0.065437
Temperature                    0.672702
Pain Assessment Method         0.709175
Activity Tolerance             0.817855
Head of Bed                    0.804002
dtype: float64

In [16]:
# 각 환자들이 최소 1번 이상 해당 라벨의 검사지표가 있는지


def calculate_label_experience_ratio(final_df, label_column='Diastolic BP'):
    # 라벨이 1번 이상 기록된 환자만 필터링
    label_count_per_patient = final_df.groupby(['stay_id', label_column]).size().reset_index(name='count')
    
    # 라벨이 1번 이상 기록된 환자만 필터링
    patients_with_label = label_count_per_patient[label_count_per_patient['count'] > 0]
    
    # 환자별 라벨이 1번 이상 존재한 것 확인 (각 환자별 라벨이 기록된 개수)
    patients_with_label_summary = patients_with_label.groupby('stay_id')[label_column].nunique()
    
    # 각 환자(stay_id)별로 1번 이상 기록된 라벨 개수 출력
    #print(patients_with_label_summary)
    
    # 1번 이상 경험한 환자 수
    num_patients_with_label = len(patients_with_label_summary)
    
    # 전체 환자 수
    num_total_patients = len(final_df['stay_id'].unique())
    
    # 1번 이상 경험한 환자 비율 계산
    patients_with_label_ratio = num_patients_with_label / num_total_patients
    
    # 비율 출력
    #print(f"{label_column} 피검 환자 비율: {patients_with_label_ratio:.2f}")
    
    # 백분율로 출력
    patients_with_label_percentage = patients_with_label_ratio * 100
    print(f"{label_column} 피검 환자 비율: {patients_with_label_percentage:.2f}%")
    
    #return patients_with_label_ratio, patients_with_label_percentage


# 'stay_id' 컬럼 제외한 나머지 컬럼 리스트 얻기: 단 stay_id와 time_bin은 제외
columns_to_check = [col for col in final_df.columns if col != 'stay_id' and col != 'time_bin']

# 각 컬럼에 대해 함수 호출
for col in columns_to_check:
    calculate_label_experience_ratio(final_df, label_column=col)

Arterial PH 피검 환자 비율: 50.87%
Blood Glucose 피검 환자 비율: 94.56%
Diastolic BP 피검 환자 비율: 99.83%
GCS_Eye 피검 환자 비율: 99.54%
GCS_Motor 피검 환자 비율: 99.43%
GCS_Verbal 피검 환자 비율: 99.48%
Heart Rate 피검 환자 비율: 99.89%
O2 Saturation Pulseoxymetry 피검 환자 비율: 99.83%
Respiratory Rate 피검 환자 비율: 99.66%
Systolic BP 피검 환자 비율: 99.84%
Temperature 피검 환자 비율: 96.67%
Pain Assessment Method 피검 환자 비율: 96.19%
Activity Tolerance 피검 환자 비율: 88.33%
Head of Bed 피검 환자 비율: 91.48%


In [92]:
#데이터 프레임 저장
#final_df.to_csv('average_calculation_per_hour_in_6hour.csv', index=False)

In [17]:
len(final_df['stay_id'].unique()) # 총 환자

72109

In [18]:
# 6시간동안 한번도 측정하지 않은 컬럼이 있는 환자는 제거 ->하나라도 있으면 보간가능
filtered_df = final_df.groupby('stay_id').filter(
    lambda group: not (group.isna().all(axis=0).any())
)
# 배열, 포인터, 구조체
# 결과 확인
filtered_df

,stay_id,time_bin,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed
0,30000153,0,NaN,NaN,77.0,3.0,5.0,1.0,104.0,100.0,17.0,113.0,NaN,Change in Vitals,Tolerated Well,Reverse T-berg
1,30000153,1,7.30,167.0,66.5,NaN,NaN,NaN,NaN,NaN,16.0,141.0,37.277778,NaN,NaN,NaN
2,30000153,2,NaN,NaN,NaN,NaN,NaN,NaN,83.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
3,30000153,3,7.31,183.5,60.0,NaN,NaN,NaN,87.5,100.0,15.0,116.0,37.500000,NaN,NaN,NaN
4,30000153,4,NaN,NaN,56.0,4.0,6.0,1.0,103.0,100.0,20.0,111.0,NaN,Non-verbal Cues,Tolerated Well,Reverse T-berg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832965,39999384,7,7.44,116.0,45.0,NaN,NaN,NaN,63.0,99.0,15.0,144.0,NaN,NaN,NaN,NaN
832966,39999384,8,NaN,NaN,46.0,4.0,6.0,5.0,70.0,100.0,14.0,155.0,36.388889,Patient Verbalized,Tolerated Well,30 Degrees
832967,39999384,9,NaN,NaN,49.0,NaN,NaN,NaN,70.0,100.0,11.0,159.0,NaN,NaN,NaN,NaN
832968,39999384,10,NaN,NaN,48.0,NaN,NaN,NaN,68.0,100.0,24.0,157.0,NaN,NaN,NaN,NaN


In [19]:
#결측치 비율 출력
filtered_df.isnull().sum() / len(filtered_df)

stay_id                        0.000000
time_bin                       0.000000
Arterial PH                    0.781590
Blood Glucose                  0.646001
Diastolic BP                   0.063198
GCS_Eye                        0.689668
GCS_Motor                      0.691566
GCS_Verbal                     0.690406
Heart Rate                     0.046559
O2 Saturation Pulseoxymetry    0.060350
Respiratory Rate               0.052771
Systolic BP                    0.063067
Temperature                    0.628223
Pain Assessment Method         0.693696
Activity Tolerance             0.800460
Head of Bed                    0.787885
dtype: float64

In [20]:
len(filtered_df['stay_id'].unique()) # 총 환자수

29032

In [21]:
#컬럼 리스트 얻기: 단 stay_id와 time_bin은 제외
columns_to_check = [col for col in filtered_df.columns if col != 'stay_id' and col != 'time_bin']

# 각 컬럼에 대해 함수 호출
for col in columns_to_check:
    calculate_label_experience_ratio(filtered_df, label_column=col)

Arterial PH 피검 환자 비율: 100.00%
Blood Glucose 피검 환자 비율: 100.00%
Diastolic BP 피검 환자 비율: 100.00%
GCS_Eye 피검 환자 비율: 100.00%
GCS_Motor 피검 환자 비율: 100.00%
GCS_Verbal 피검 환자 비율: 100.00%
Heart Rate 피검 환자 비율: 100.00%
O2 Saturation Pulseoxymetry 피검 환자 비율: 100.00%
Respiratory Rate 피검 환자 비율: 100.00%
Systolic BP 피검 환자 비율: 100.00%
Temperature 피검 환자 비율: 100.00%
Pain Assessment Method 피검 환자 비율: 100.00%
Activity Tolerance 피검 환자 비율: 100.00%
Head of Bed 피검 환자 비율: 100.00%


In [22]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 337100 entries, 0 to 832969
Data columns (total 16 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   stay_id                      337100 non-null  int64  
 1   time_bin                     337100 non-null  int64  
 2   Arterial PH                  73626 non-null   float64
 3   Blood Glucose                119333 non-null  float64
 4   Diastolic BP                 315796 non-null  float64
 5   GCS_Eye                      104613 non-null  float64
 6   GCS_Motor                    103973 non-null  float64
 7   GCS_Verbal                   104364 non-null  float64
 8   Heart Rate                   321405 non-null  float64
 9   O2 Saturation Pulseoxymetry  316756 non-null  float64
 10  Respiratory Rate             319311 non-null  float64
 11  Systolic BP                  315840 non-null  float64
 12  Temperature                  125326 non-null  float64
 13  Pain

In [23]:
# 결측치 전진, 후진 보간

grouped = filtered_df.groupby('stay_id')

def fill_missing_values(group):
    # 기존 인덱스를 리셋하여 그룹화 키를 제거합니다
    group = group.reset_index(drop=True)

    # 'time_bin'을 인덱스로 설정합니다
    group = group.set_index('time_bin')

    # 1. 모든 변수의 결측치를 전진 채움으로 채웁니다
    group = group.ffill()

    # 2. 여전히 결측치가 남아있는 경우 후진 채움으로 채웁니다
    group = group.bfill()

    # 인덱스를 리셋합니다
    group = group.reset_index()

    return group

# 그룹화된 데이터에 결측치 처리 함수를 적용합니다
filtered_df = grouped.apply(fill_missing_values).reset_index(drop=True)

# 결과를 확인합니다
filtered_df

C:\Users\Eugene Clark\AppData\Local\Temp\ipykernel_17396\1690401265.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = grouped.apply(fill_missing_values).reset_index(drop=True)


,time_bin,stay_id,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed
0,0,30000153,7.30,167.0,77.0,3.0,5.0,1.0,104.0,100.0,17.0,113.0,37.277778,Change in Vitals,Tolerated Well,Reverse T-berg
1,1,30000153,7.30,167.0,66.5,3.0,5.0,1.0,104.0,100.0,16.0,141.0,37.277778,Change in Vitals,Tolerated Well,Reverse T-berg
2,2,30000153,7.30,167.0,66.5,3.0,5.0,1.0,83.0,100.0,16.0,141.0,37.277778,Change in Vitals,Tolerated Well,Reverse T-berg
3,3,30000153,7.31,183.5,60.0,3.0,5.0,1.0,87.5,100.0,15.0,116.0,37.500000,Change in Vitals,Tolerated Well,Reverse T-berg
4,4,30000153,7.31,183.5,56.0,4.0,6.0,1.0,103.0,100.0,20.0,111.0,37.500000,Non-verbal Cues,Tolerated Well,Reverse T-berg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337095,7,39999384,7.44,116.0,45.0,4.0,6.0,5.0,63.0,99.0,15.0,144.0,36.888889,Patient Verbalized,Tolerated Well,30 Degrees
337096,8,39999384,7.44,116.0,46.0,4.0,6.0,5.0,70.0,100.0,14.0,155.0,36.388889,Patient Verbalized,Tolerated Well,30 Degrees
337097,9,39999384,7.44,116.0,49.0,4.0,6.0,5.0,70.0,100.0,11.0,159.0,36.388889,Patient Verbalized,Tolerated Well,30 Degrees
337098,10,39999384,7.44,116.0,48.0,4.0,6.0,5.0,68.0,100.0,24.0,157.0,36.388889,Patient Verbalized,Tolerated Well,30 Degrees


In [24]:
len(filtered_df['stay_id'].unique()) # 총 환자수

29032

In [25]:
filtered_df.to_csv('circulatory_diseases_dataset_12.csv', index=False, encoding='utf-8-sig')

In [27]:
#환자의 나이 및 성별 포함
patients = patients[['subject_id', 'gender', 'anchor_age']]
personal = pd.merge(icustays[['subject_id','stay_id']],patients,on = 'subject_id')
personal = personal[['stay_id','gender','anchor_age']]
personal

,stay_id,gender,anchor_age
0,34569476,F,67
1,34569476,F,67
2,39289362,F,49
3,39289362,F,49
4,32563675,F,72
...,...,...,...
48383,39988534,F,48
48384,39988873,F,42
48385,39988873,F,42
48386,39992247,M,72


In [28]:
# 환자 개인정보와 바이탈 지표 병합
#최종 데이터셋인 dataset
dataset = pd.merge(filtered_df,personal)
dataset

,time_bin,stay_id,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed,gender,anchor_age
0,0,30000213,7.37,125.0,62.5,2.0,4.0,1.0,79.5,96.5,20.5,165.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
1,1,30000213,7.37,108.0,53.0,2.0,4.0,1.0,66.0,97.0,23.0,126.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
2,2,30000213,7.37,126.5,67.0,4.0,6.0,4.0,83.0,100.0,18.0,118.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
3,3,30000213,7.37,126.5,57.0,4.0,6.0,4.0,75.0,99.0,26.0,128.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
4,4,30000213,7.37,126.5,56.0,4.0,6.0,4.0,79.0,98.0,20.0,130.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205115,9,39999301,7.42,127.0,68.0,4.0,6.0,5.0,59.0,94.0,14.0,120.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,M,77
205116,10,39999301,7.38,128.0,71.0,4.0,6.0,5.0,61.0,94.0,15.0,126.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,M,77
205117,10,39999301,7.38,128.0,71.0,4.0,6.0,5.0,61.0,94.0,15.0,126.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,M,77
205118,11,39999301,7.38,128.0,58.0,4.0,6.0,5.0,57.0,95.0,11.0,110.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,M,77


In [29]:
# 성별 인코딩
d = {'M' : 1 , 'F': 0 }
dataset['gender'] = dataset['gender'].map(d)
dataset

,time_bin,stay_id,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed,gender,anchor_age
0,0,30000213,7.37,125.0,62.5,2.0,4.0,1.0,79.5,96.5,20.5,165.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
1,1,30000213,7.37,108.0,53.0,2.0,4.0,1.0,66.0,97.0,23.0,126.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
2,2,30000213,7.37,126.5,67.0,4.0,6.0,4.0,83.0,100.0,18.0,118.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
3,3,30000213,7.37,126.5,57.0,4.0,6.0,4.0,75.0,99.0,26.0,128.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
4,4,30000213,7.37,126.5,56.0,4.0,6.0,4.0,79.0,98.0,20.0,130.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205115,9,39999301,7.42,127.0,68.0,4.0,6.0,5.0,59.0,94.0,14.0,120.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,1,77
205116,10,39999301,7.38,128.0,71.0,4.0,6.0,5.0,61.0,94.0,15.0,126.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,1,77
205117,10,39999301,7.38,128.0,71.0,4.0,6.0,5.0,61.0,94.0,15.0,126.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,1,77
205118,11,39999301,7.38,128.0,58.0,4.0,6.0,5.0,57.0,95.0,11.0,110.0,36.000000,Patient Verbalized,Tolerated Well,45 Degrees,1,77


In [30]:
#결측치 비율 출력
dataset.isnull().sum() / len(filtered_df)

time_bin                       0.0
stay_id                        0.0
Arterial PH                    0.0
Blood Glucose                  0.0
Diastolic BP                   0.0
GCS_Eye                        0.0
GCS_Motor                      0.0
GCS_Verbal                     0.0
Heart Rate                     0.0
O2 Saturation Pulseoxymetry    0.0
Respiratory Rate               0.0
Systolic BP                    0.0
Temperature                    0.0
Pain Assessment Method         0.0
Activity Tolerance             0.0
Head of Bed                    0.0
gender                         0.0
anchor_age                     0.0
dtype: float64

In [31]:
filtered_df = dataset
filtered_df.to_csv('circulatory_diseases_dataset_12.csv', index=False, encoding='utf-8-sig')

In [32]:
import pandas as pd

# 1. circulatory_diseases_dataset.csv 파일 읽기
circulatory_df = pd.read_csv('circulatory_diseases_dataset_12.csv')

# 2. patients_icu.csv 파일 읽기
icu_df = pd.read_csv('content/patients_icu.csv')

# 3. 두 데이터셋을 stay_id 컬럼을 기준으로 조인
merged_df = pd.merge(circulatory_df[['stay_id']], icu_df[['stay_id', 'los']], on='stay_id', how="inner")

# 4. stay_id와 los 컬럼만 선택
result_df = merged_df[['stay_id', 'los']]

# 결과 출력
print(result_df)

         stay_id       los
0       30000213  1.635278
1       30000213  1.635278
2       30000213  1.635278
3       30000213  1.635278
4       30000213  1.635278
...          ...       ...
158346  39999301  1.146910
158347  39999301  1.146910
158348  39999301  1.146910
158349  39999301  1.146910
158350  39999301  1.146910

[158351 rows x 2 columns]


In [33]:
len(result_df['stay_id'].unique()) # 총 환자수

9460

In [34]:
#결측치 비율 출력
result_df.isnull().sum() / len(result_df)

stay_id    0.0
los        0.0
dtype: float64

In [35]:
# 타켓 데이터셋 추출
result_df.to_csv('circulatory_target_in_12hour.csv', index=False, encoding='utf-8-sig')
print(result_df)

         stay_id       los
0       30000213  1.635278
1       30000213  1.635278
2       30000213  1.635278
3       30000213  1.635278
4       30000213  1.635278
...          ...       ...
158346  39999301  1.146910
158347  39999301  1.146910
158348  39999301  1.146910
158349  39999301  1.146910
158350  39999301  1.146910

[158351 rows x 2 columns]


In [36]:
# los값이 결측치인 환자 제거 후 csv 추출
filtered_circulatory_df = circulatory_df[circulatory_df['stay_id'].isin(result_df['stay_id'])]

print(filtered_circulatory_df)
filtered_circulatory_df.to_csv('circulatory_dataset_in_12hour.csv', index=False, encoding='utf-8-sig')

        time_bin   stay_id  Arterial PH  Blood Glucose  Diastolic BP  GCS_Eye  \
0              0  30000213         7.37          125.0          62.5      2.0   
1              1  30000213         7.37          108.0          53.0      2.0   
2              2  30000213         7.37          126.5          67.0      4.0   
3              3  30000213         7.37          126.5          57.0      4.0   
4              4  30000213         7.37          126.5          56.0      4.0   
...          ...       ...          ...            ...           ...      ...   
205115         9  39999301         7.42          127.0          68.0      4.0   
205116        10  39999301         7.38          128.0          71.0      4.0   
205117        10  39999301         7.38          128.0          71.0      4.0   
205118        11  39999301         7.38          128.0          58.0      4.0   
205119        11  39999301         7.38          128.0          58.0      4.0   

        GCS_Motor  GCS_Verb

In [37]:
len(filtered_circulatory_df['stay_id'].unique()) # 총 환자수

9460

In [38]:
chart_df = filtered_circulatory_df

# 고유값 조회
#o2_device_unique = chart_df['O2 Delivery Device(s)'].unique()
pain_assessment_method_unique = chart_df['Pain Assessment Method'].unique()
#pain_location_unique = chart_df['Pain Location'].unique()
activity_tolerance_unique = chart_df['Activity Tolerance'].unique()
#heart_rhythm_unique = chart_df['Heart Rhythm'].unique()
head_of_bed_unique = chart_df['Head of Bed'].unique()

# 결과 출력
#print("O2 Delivery Device(s) - Unique Values:", o2_device_unique)
print("Pain Assessment Method - Unique Values:", pain_assessment_method_unique)
#print("Pain Location - Unique Values:", pain_location_unique)
print("Activity Tolerance - Unique Values:", activity_tolerance_unique)
#print("Heart Rhythm - Unique Values:", heart_rhythm_unique)
print("Head of Bed - Unique Values:", head_of_bed_unique)

Pain Assessment Method - Unique Values: ['Non-verbal Cues' 'Patient Verbalized' 'Change in Vitals' 'Grimace Scale'
 'Not applicable' 'FLACC' '---' 'Wong-Baker Scale' 'PAINAD']
Activity Tolerance - Unique Values: ['Tolerated Well' 'Good' 'Fair' 'Poor']
Head of Bed - Unique Values: ['30 Degrees' '90 Degrees' '45 Degrees' 'Flat in Bed' 'Reverse T-berg'
 '15 Degrees' '---' 'Trendelenberg']


In [39]:
import pandas as pd

chart_df = filtered_circulatory_df.copy()  # 원본 데이터 유지
# 컬럼 이름을 리스트로 저장
columns_to_clean = ['Pain Assessment Method',
                    'Activity Tolerance', 'Head of Bed']

# 공백 제거 및 전진/후진 채움 적용
for column in columns_to_clean:
    # 공백 제거 (양쪽 공백)
    chart_df[column] = chart_df[column].str.strip()
    
    # '---' 값을 NaN으로 대체 (결측치 처리)
    chart_df[column] = chart_df[column].replace('---', pd.NA)
    
    # 전진 채움 적용
    chart_df[column] = chart_df[column].ffill()
    
    # 후진 채움 적용
    chart_df[column] = chart_df[column].bfill()

# 결과 확인
print(chart_df[columns_to_clean].head())

  Pain Assessment Method Activity Tolerance Head of Bed
0        Non-verbal Cues     Tolerated Well  30 Degrees
1        Non-verbal Cues     Tolerated Well  30 Degrees
2        Non-verbal Cues     Tolerated Well  30 Degrees
3        Non-verbal Cues     Tolerated Well  30 Degrees
4        Non-verbal Cues     Tolerated Well  30 Degrees


In [40]:
# 고유값 조회
#o2_device_unique = chart_df['O2 Delivery Device(s)'].unique()
pain_assessment_method_unique = chart_df['Pain Assessment Method'].unique()
#pain_location_unique = chart_df['Pain Location'].unique()
activity_tolerance_unique = chart_df['Activity Tolerance'].unique()
#heart_rhythm_unique = chart_df['Heart Rhythm'].unique()
head_of_bed_unique = chart_df['Head of Bed'].unique()

# 결과 출력
#print("O2 Delivery Device(s) - Unique Values:", o2_device_unique)
print("Pain Assessment Method - Unique Values:", pain_assessment_method_unique)
#print("Pain Location - Unique Values:", pain_location_unique)
print("Activity Tolerance - Unique Values:", activity_tolerance_unique)
#print("Heart Rhythm - Unique Values:", heart_rhythm_unique)
print("Head of Bed - Unique Values:", head_of_bed_unique)

Pain Assessment Method - Unique Values: ['Non-verbal Cues' 'Patient Verbalized' 'Change in Vitals' 'Grimace Scale'
 'Not applicable' 'FLACC' 'Wong-Baker Scale' 'PAINAD']
Activity Tolerance - Unique Values: ['Tolerated Well' 'Good' 'Fair' 'Poor']
Head of Bed - Unique Values: ['30 Degrees' '90 Degrees' '45 Degrees' 'Flat in Bed' 'Reverse T-berg'
 '15 Degrees' 'Trendelenberg']


In [41]:
# 최종 6시간 데이터셋
chart_df.to_csv('circulatory_dataset_in_12hour.csv', index=False, encoding='utf-8-sig')

In [42]:
# 마지막 점검: 환자수
len(chart_df['stay_id'].unique()) # 총 환자수

9460

In [44]:
df111 = pd.read_csv("circulatory_target_in_12hour.csv")

# stay_id와 los 컬럼이 같은 행을 제거
df111_unique = df111.drop_duplicates(subset=['stay_id', 'los'])

print(df111_unique)
df111_unique.to_csv('circulatory_target_in_12hour1.csv', index=False, encoding='utf-8-sig')

         stay_id       los
0       30000213  1.635278
12      30001396  1.167627
24      30002415  1.014560
36      30002548  1.233889
47      30004798  0.820937
...          ...       ...
158243  39988873  1.590185
158267  39989733  4.054572
158279  39991309  4.103171
158315  39993476  1.025810
158327  39999301  1.146910

[9460 rows x 2 columns]
